<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект. Анализ вакансий на HeadHunter

## (1) Работа с базой данных из Python

---

В реальной деятельности дата-сайентисту требуется не только писать запрос к данным, но и затем обрабатывать его результаты с помощью *Python*. Для решения таких задач требуется некоторое средство, которое будет связывать *Python* и *PostgreSQL* так, чтобы мы могли с помощью *Python* отправлять запросы в *Postgres* и принимать оттуда результаты.

Именно таким средством является пакет [psycopg2](https://www.psycopg.org/docs/), о котором мы сейчас и поговорим.

## Установка psycopg2

```linux
pip install psycopg2
```

Для подключения нам потребуются следующие данные:

* dbname — название базы, к которой нужно подключиться;
* user — имя пользователя в СУБД;
* password — пароль;
* host — адрес, по которому нужно подключиться;
* port — порт, к которому нужно подключиться (по умолчанию равен 5432).

Для нашей базы эти параметры сохранены в текущей директории в файле `config.ini`, который добавлен в список исключений `.gitignore` 

In [29]:
import configparser
import pandas as pd
import psycopg2
import warnings

In [30]:
# Исключение предупреждающих сообщений из выходных данных ячеек
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [31]:
# Получение сведений об авторизации. Установление содинения с БД 
config = configparser.ConfigParser()
config.read("config.ini")
connection = psycopg2.connect(
    dbname=config.get("postgresql", 'DBNAME'),
    user=config.get("postgresql", "USER"),
    host=config.get("postgresql", "HOST"),
    password=config.get("postgresql", "PASSWORD"),
    port=config.get("postgresql", "PORT"),
)

Запрос:

In [15]:
# n = 10
# query = f"""
#     select *
#     from vacancies
#     limit {n}
# """

Выполнение запроса

In [14]:
# df = pd.read_sql_query(query, connection)
# connection.close()
# df

## (2) Введение. Знакомство с данными
---

Представьте, что вы устроились на работу в кадровое агентство, которое подбирает вакансии для IT-специалистов. Ваш первый проект — создание модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist. Сначала вам необходимо понять, что из себя представляют данные и насколько они соответствуют целям проекта. В литературе эта часть работы над ML-проектом называется Data Understanding, или анализ данных.

Наш проект включает в себя несколько этапов:

* знакомство с данными;
* предварительный анализ данных;
* детальный анализ вакансий;
* анализ работодателей;
* предметный анализ.

Каждая из частей будет состоять из блока практических заданий, которые вам необходимо выполнить в своих Jupyter-ноутбуках, и контрольных вопросов на платформе, которые проверяются автоматически.

Также вам предстоит отправить свой код ментору для код-ревью. Вам будет предоставлен ноутбук-шаблон и требования, согласно которым вы должны оформить своё решение.

Требования к оформлению ноутбука-решения:

* Решение оформляется только в Jupyter Notebook.
* Решение оформляется в соответствии с [ноутбуком-шаблоном](https://lms.skillfactory.ru/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block@Project_2_%D0%9D%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA_%D1%88%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD.ipynb).
* Каждое задание выполняется в отдельной ячейке, выделенной под задание (в шаблоне они помечены как ваш код здесь). Не следует создавать много ячеек для решения задачи — это провоцирует неудобства при проверке.
* Текст SQL-запросов и код на Python должны быть читаемыми. Не забывайте про отступы в SQL-коде.
* Выводы по каждому этапу оформляются в формате Markdown в отдельной ячейке (в шаблоне они помечены как ваши выводы здесь).
* Выводы можно дополнительно проиллюстрировать с помощью графиков. Они оформляются в соответствии с теми правилами, которые мы приводили в модуле по визуализации данных.
* Не забудьте удалить ячейку с данными соединения перед фиксацией работы в GitHub.



### Знакомство с данными

Все необходимые таблицы находятся в схеме public базы данных project_sql (именно эту базу вам необходимо указать в параметре dbname при подключении).

То есть параметры подключения будут следующими: см. файл `config.ini`

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_1.png)

Познакомимся с каждой таблицей.

vacancies

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_2.png)


Зарплатная вилка — это верхняя и нижняя граница оплаты труда в рублях (зарплаты в других валютах уже переведены в рубли). Соискателям она показывает, в каком диапазоне компания готова платить сотруднику на этой должности.

areas

Таблица-справочник, которая хранит код региона и его название.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_3.png)

employers

Таблица-справочник со списком работодателей.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_4.png)

industries

Таблица-справочник вариантов сфер деятельности работодателей.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_5.png)

employers_industries

Дополнительная таблица, которая существует для организации связи между работодателями и сферами их деятельности.

Эта таблица нужна нам, поскольку у одного работодателя может быть несколько сфер деятельности (или работодатели могут вовсе не указать их). Для удобства анализа необходимо хранить запись по каждой сфере каждого работодателя в отдельной строке таблицы.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_6.png)

## (3) Предварительный анализ данных
---

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''
    select 
        count(*)
    from
        vacancies
'''
df = pd.read_sql_query(query_3_1, connection)

In [6]:
# результат запроса
print(f"В базе содержится {df.iloc[0, 0]} вакансий.")

В базе содержится 49197 вакансий.


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [32]:
# текст запроса
query_3_2 = f'''
    select 
        count(*)
    from
        employers
'''
df = pd.read_sql_query(query_3_2, connection)

In [33]:
# результат запроса
print(f"В базе содержится {df.iloc[0, 0]} работодателей.")

В базе содержится 23501 работодателей.


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [35]:
# текст запроса
query_3_3 = f'''
    select 
        count(*)
    from
        areas
'''
df = pd.read_sql_query(query_3_3, connection)

In [37]:
# результат запроса
print(f"В базе содержится {df.iloc[0, 0]} региона.")

В базе содержится 1362 региона.


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [38]:
# текст запроса
query_3_4 = f'''
    select 
        count(*)
    from
        industries
'''
df = pd.read_sql_query(query_3_4, connection)

In [40]:
# результат запроса
print(f"В базе содержится {df.iloc[0, 0]} сферы деятельности.")

В базе содержится 294 сферы деятельности.


## (4) Детальный анализ ваканский
---

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [32]:
# текст запроса
query_4_1 = f'''
    select
        a.name as area,
        count(v.id) as cnt
    from
        areas as a
        join vacancies as v on a.id = v.area_id
    group by
        a.id
    order by
        2 desc
    limit
        5
'''
df = pd.read_sql_query(query_4_1, connection)

In [33]:
print("Количество вакансий по регионам РФ (5-ка лидеров)")
display(df)

Количество вакансий по регионам РФ (5-ка лидеров)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [46]:
# текст запроса
query_4_2 = f'''
    select
        count(*)
    from
        vacancies as v
    where
        v.salary_from is not null
        or v.salary_to is not null
'''
df = pd.read_sql_query(query_4_2, connection)

In [47]:
print(f"Количество вакансий, где заполнено хотя бы одно из полей с зарплатой составляет {df.iloc[0, 0]}.")

Количество вакансий, где заполнено хотя бы одно из полей с зарплатой составляет 24073.


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [72]:
# текст запроса
query_4_3_1 = f'''
    select
        round(avg(salary_from))
    from
        vacancies
'''
query_4_3_2 = f'''
    select
        round(avg(salary_to))
    from
        vacancies
'''

salary_from_mean = int(pd.read_sql_query(query_4_3_1, connection).iloc[0, 0])
salary_to_mean = int(pd.read_sql_query(query_4_3_2, connection).iloc[0, 0])

In [79]:
print("Среднее значение зарплаты:")
print(f"    \u2022 для нижней границы составляет {salary_from_mean:,} \u20BD;".replace(",", " "))
print(f"    \u2022 для верхней границы составляет {salary_to_mean:,} \u20BD.".replace(",", " "))

Среднее значение зарплаты:
    • для нижней границы составляет 71 065 ₽;
    • для верхней границы составляет 110 537 ₽.


In [73]:
salary_to_mean

110537